In [39]:
import pandas as pd
import numpy as np

In [40]:
df = pd.read_csv('TD_HOSPITAL_TRAIN.csv')

In [41]:
df.drop(['dose','pdeath'],axis=1)
for index in range(len(df['sex'])):
    val = df['sex'][index]
    if val == "male" or val == "Male" or val == "M" or val == 1:
        df['sex'][index] = 0
    else:
        df['sex'][index] = 1

# print(df['sex'])

def attempt_convert_to_float(column):
    try:
        return column.astype(float)
    except ValueError:  # If conversion fails, return the original column
        return column

# Apply the function to each column
df = df.apply(attempt_convert_to_float)

for col in df.columns:
    if df.dtypes[col] == "float64":
        df[col].fillna(df[col].mean(), inplace=True)

# df = pd.get_dummies(df, columns=['sex', 'race', 'primary'], drop_first=True)
df = pd.get_dummies(df, columns=['dnr','race', 'primary', 'disability', 'income' ,'extraprimary', 'cancer' ], drop_first=True)

df.to_csv('output.csv')

/var/folders/6w/qt75n21508bdvzq5z1lgl6sr0000gn/T/ipykernel_22759/3518595214.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sex'][index] = 0
/var/folders/6w/qt75n21508bdvzq5z1lgl6sr0000gn/T/ipykernel_22759/3518595214.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sex'][index] = 1


In [42]:

correlation = df.corr()['death'].sort_values(ascending=False)
# print(correlation)
# threshold = float(input("Enter the threshold value you would like to analyze:"))
# print(threshold)

threshold = 0.3

# Identify features that have a correlation magnitude above the threshold
significant_features = correlation[correlation.abs() > threshold].index.tolist()

# Exclude the target variable 'death' from the features list
significant_features.remove('death')

new_df = df[significant_features + ['death']]

new_df.to_csv('sig.csv')


In [46]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

df = pd.read_csv('sig.csv')

# Split data into features and target variable
X = df.drop('death', axis=1)
y = df['death']

# Standardize numerical features
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

regulizer = tf.keras.regularizers.l2(0.001)
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(200, activation='relu', kernel_regularizer=regulizer),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

opt = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(opt, 
              loss='binary_crossentropy', 
              metrics=['accuracy'])


history = model.fit(X_train, y_train, 
                    epochs=100, 
                    batch_size=60, 
                    validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Sample prediction
sample_data = X_test.iloc[0]  # get the first row from the test set
predicted_prob = model.predict(np.array([sample_data]))[0][0]
# print(f"The probability of death is: {predicted_prob * 100:.2f}%")
# predicted_label = (predicted_prob > 0.5).astype(int)
# print(f"Predicted Label: {predicted_label}")

model.save('Mynewmodel.h5')



Epoch 1/100
83/83 [==============================] - 0s 2ms/step - loss: 0.3738 - accuracy: 0.8310 - val_loss: 0.3352 - val_accuracy: 0.8470
Epoch 2/100
83/83 [==============================] - 0s 1ms/step - loss: 0.3545 - accuracy: 0.8381 - val_loss: 0.3333 - val_accuracy: 0.8503
Epoch 3/100
83/83 [==============================] - 0s 1ms/step - loss: 0.3537 - accuracy: 0.8391 - val_loss: 0.3243 - val_accuracy: 0.8565
Epoch 4/100
83/83 [==============================] - 0s 1ms/step - loss: 0.3509 - accuracy: 0.8358 - val_loss: 0.3410 - val_accuracy: 0.8588
Epoch 5/100
83/83 [==============================] - 0s 1ms/step - loss: 0.3492 - accuracy: 0.8395 - val_loss: 0.3284 - val_accuracy: 0.8565
Epoch 6/100
83/83 [==============================] - 0s 1ms/step - loss: 0.3539 - accuracy: 0.8364 - val_loss: 0.3292 - val_accuracy: 0.8551
Epoch 7/100
83/83 [==============================] - 0s 1ms/step - loss: 0.3467 - accuracy: 0.8372 - val_loss: 0.3352 - val_accuracy: 0.8546
Epoch 8/100
8

1/1 [==============================] - 0s 27ms/step


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
class Solution:
    def __init__(self):
        #Initialize any global variables here
        self.model = tf.keras.models.load_model('Mynewmodel.h5')

    def calculate_death_prob(self, data):
        
        """
        This function should return your final prediction!
        """
        df = pd.DataFrame(data)
        df.drop(['dose','pdeath'],axis=1)
        for index in range(len(df['sex'])):
            val = df['sex'][index]
            if val == "male" or val == "Male" or val == "M" or val == 1:
                df['sex'][index] = 0
            else:
                df['sex'][index] = 1

        # print(df['sex'])

        def attempt_convert_to_float(column):
            try:
                return column.astype(float)
            except ValueError:  # If conversion fails, return the original column
                return column

        # Apply the function to each column
        df = df.apply(attempt_convert_to_float)

        for col in df.columns:
            if df.dtypes[col] == "float64":
                df[col].fillna(df[col].mean(), inplace=True)

        # df = pd.get_dummies(df, columns=['sex', 'race', 'primary'], drop_first=True)
        df = pd.get_dummies(df, columns=['dnr','race', 'primary', 'disability', 'income' ,'extraprimary', 'cancer' ], drop_first=True)
        correlation = df.corr()['death'].sort_values(ascending=False)
        # print(correlation)
        # threshold = float(input("Enter the threshold value you would like to analyze:"))
        # print(threshold)

        threshold = 0.05

        # Identify features that have a correlation magnitude above the threshold
        significant_features = correlation[correlation.abs() > threshold].index.tolist()

        # Exclude the target variable 'death' from the features list
        significant_features.remove('death')

        new_df = df[significant_features]
        prediction = self.model.predict(new_df.to_numpy())
        return float(prediction[0][0])
        
        


In [49]:
from flask import Flask, jsonify, request
import tensorflow as tf
import pandas as pd
import random
app = Flask(__name__)
@app.route("/death_probability", methods=["POST"])
def q1():
    solution = Solution()
    data = request.get_json()
    return {
        "probability": solution.calculate_death_prob(data)}

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5555)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5555
 * Running on http://10.50.192.142:5555
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5555
 * Running on http://10.50.192.142:5555
Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit
[2023-10-28 19:02:42,341] ERROR in app: Exception on /death_probability [POST]
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/flask/app.py", line 1455, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/flask/app.py", line 869, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.fram